In [1]:
import xarray as xr
import numpy as np
from statsmodels.stats.multitest import fdrcorrection


In [7]:
#Apply FDR to t2m/blocking correlations

pztO=xr.open_dataset('t2m_pvals_monthlyall_climsmoothed_Oampl2.nc')
pzbO=xr.open_dataset('block_pvals_monthlyall_climsmoothed_Oampl2.nc')

pztM=xr.open_dataset('t2m_pvals_monthlyall_climsmoothed_Mampl2.nc')
pzbM=xr.open_dataset('block_pvals_monthlyall_climsmoothed_Mampl2.nc')

pzto_st=pztO.stack(z={'latitude','longitude'})
pnew_to=pzto_st.t2m.copy(deep=True)
pnew_to.values=fdrcorrection(pnew_to.values,alpha=0.15, method='indep', is_sorted=False)[1]

pztm_st=pztM.stack(z={'latitude','longitude'})
pztm_drna=pztm_st.dropna(dim='z')
pnew_tm=pztm_st.t2m.copy(deep=True)
pnew_drnatm=pztm_drna.t2m.copy(deep=True)
pzfdrst=fdrcorrection(pztm_drna.t2m.values,alpha=0.15, method='indep', is_sorted=False)[1]
pnew_drnatm.values=pzfdrst
pnew_tm.loc[pnew_tm.z.isin(pnew_drnatm.z)]=pnew_drnatm

pzbo_st=pzbO.stack(z={'latitude','longitude'})
pzbo_drna=pzbo_st.dropna(dim='z')
pnew_bo=pzbo_st.block.copy(deep=True)
pnew_drnabo=pzbo_drna.block.copy(deep=True)
pzfdrbo=fdrcorrection(pzbo_drna.block.values,alpha=0.15, method='indep', is_sorted=False)[1]
pnew_drnabo.values=pzfdrbo
pnew_bo.loc[pnew_bo.z.isin(pnew_drnabo.z)]=pnew_drnabo

pzbm_st=pzbM.stack(z={'latitude','longitude'})
pzbm_drna=pzbm_st.dropna(dim='z')
pnew_bm=pzbm_st.block.copy(deep=True)
pnew_drnabm=pzbm_drna.block.copy(deep=True)
pzfdrbm=fdrcorrection(pzbm_drna.block.values,alpha=0.15, method='indep', is_sorted=False)[1]
pnew_drnabm.values=pzfdrbm
pnew_bm.loc[pnew_bm.z.isin(pnew_drnabm.z)]=pnew_drnabm

In [10]:
#Save to files
pnew_to.unstack().to_netcdf('t2m_pvals_monthlyall_climsmoothed_Oampl_fdr2.nc')
pnew_bo.unstack().to_netcdf('block_pvals_monthlyall_climsmoothed_Oampl_fdr2.nc')

pnew_tm.unstack().to_netcdf('t2m_pvals_monthlyall_climsmoothed_Mampl_fdr2.nc')
pnew_bm.unstack().to_netcdf('block_pvals_monthlyall_climsmoothed_Mampl_fdr2.nc')

In [ ]:
#Apply fdr to MJO composites
sigs=[]
for j in range(8):
    sig=xr.open_dataset('p-vals_mjo_phase'+str(j+1)+'1000x.nc')
    sig_st=sig.stack(z={'latitude','longitude'})
    sig_new=sig_st.q.copy(deep=True)
    sig_new.values=fdrcorrection(sig_new.values,alpha=0.25, method='indep', is_sorted=False)[1]
    sig_new.unstack().to_netcdf('p-vals_mjo_phase'+str(j+1)+'_1000x_fdr.nc')
    sigs.append(sig_new)

In [ ]:
#Apply FDR to ENSO/SSW composites
vars1=['elnino','lanina','neutral','ssw']
for j in vars1:
    sig=xr.open_dataset('pval_'+j+'.nc')
    sig_st=sig.stack(z={'latitude','longitude'})
    sig_new=sig_st.q.copy(deep=True)
    sig_new.values=fdrcorrection(sig_new.values,alpha=0.05, method='indep', is_sorted=False)[1]
    sig_new.unstack().to_netcdf('p-val_'+j+'_fdr.nc')